In [1]:
import pandas as pd
import numpy as np

In [2]:
folder = "C:\\Users\\fpicaso\\Repos\\EUV\\"

In [10]:
TR_Log = pd.read_csv(folder + "Identified_History_NewCount_of_TR3400_2023.csv")
TR_Log["Wafer Start Time"] = pd.to_datetime(TR_Log["Wafer Start Time"])
TR_Log["Wafer End Time"] = pd.to_datetime(TR_Log["Wafer End Time"])
TR_Log["Lot Start Time"] = pd.to_datetime(TR_Log["Lot Start Time"])
TR_Log["Lot End Time"] = pd.to_datetime(TR_Log["Lot End Time"])
TR_Log["Exposure Start Time"] = pd.to_datetime(TR_Log["Exposure Start Time"])
TR_Log["Exposure End Time"] = pd.to_datetime(TR_Log["Exposure End Time"])
TR_Log["DT_FOUP"] = pd.to_datetime(TR_Log["DT_FOUP"])
TR_Log["DT_Lot"] = pd.to_datetime(TR_Log["DT_Lot"])

#Changed Type
TR_Log["Tool Name"] = TR_Log["Tool Name"].str.replace('LITHIUS PRO Z 2G','3400')
TR_Log["Tool Name"] = TR_Log["Tool Name"].str.replace('LITHIUS PRO Z','3300')

#Filtered Rows
TR_Log = TR_Log[
    (TR_Log["KD_Final"] != "")
    & (TR_Log["KD_Final"].notnull())
]

#Added Custom
TR_Log["Exposure"] = np.where((TR_Log["Exposure Start Time"].isnull()),False,True)

#Removed Columns1
columnsToDrop = [
    "Lot Name", 
    "Job Name", 
    "Wafer Result", 
    "Lot Start Time", 
    "Lot End Time", 
    "Recipe class", 
    "Recipe", 
    "Load Port", 
    "FOUP", 
    "Slot", 
    "Exposure Start Time", 
    "Exposure End Time", 
    "Lot regexp", 
    "KD regexp", 
    "DT_FOUP", 
    "DT_Lot", 
    "Lot", 
    "Route", 
    "Lot_Final"
]
TR_Log.drop(columns=columnsToDrop,inplace=True)

#Renamed Columns
TR_Log = TR_Log.rename(
    columns={
        "Tool Name": "Cluster",
        "Wafer Start Time": "Start",
        "Wafer End Time": "Finish",
        "KD_Final": "KD",
        "Duration [h]": "Duration"
    }
)

Actual_TR_Logs = pd.DataFrame(columns=[
   'Cluster', 
    'Start', 
    'Finish', 
    'Wafer', 
    'WBS', 
    'KD', 
    'Duration',
    'SC Processing [h]', 
    'Duration_NormWait [h]', 
    'Exposure', 
    'batch' 
])

clusters = TR_Log.Cluster.unique()

for cluster in clusters:
    data = TR_Log[TR_Log["Cluster"] == cluster]
    grouped = DefineBatches(TR_Log)
    Actual_TR_Logs = pd.concat([Actual_TR_Logs,grouped])

Actual_TR_Logs.head()

C:\Users\fpicaso\AppData\Local\Temp/ipykernel_12084/2777976995.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_start_filter["batch"] = index


,Cluster,Start,Finish,Wafer,WBS,KD,Duration,SC Processing [h],Duration_NormWait [h],Exposure,batch,Usage
0,3400,2023-01-02 12:17:54,2023-01-02 12:22:50,NaN,NaN,31811.0,NaN,NaN,NaN,False,1.0,0.067222
1,3400,2023-01-02 12:35:59,2023-01-02 12:51:56,NaN,NaN,31811.0,NaN,NaN,NaN,False,2.0,0.155833
2,3400,2023-01-02 14:53:34,2023-01-02 15:00:08,NaN,NaN,31811.0,NaN,NaN,NaN,False,3.0,0.109444
3,3400,2023-01-02 15:24:03,2023-01-02 15:30:36,NaN,NaN,31811.0,NaN,NaN,NaN,False,4.0,0.109167
4,3400,2023-01-02 15:51:08,2023-01-02 15:57:41,NaN,NaN,31811.0,NaN,NaN,NaN,False,5.0,0.109167


In [8]:
#DefineBatches
def DefineBatches(df):
    x = df.copy()
    SortedRows = x.sort_values(["Start"])
    index = range(0,len(SortedRows))

    #Added Index
    SortedRows["Index"] = index

    #State_DOWN_Add_Index
    SortedRows["Finish_DOWN"] = SortedRows["Finish"].shift(1)

    #Added Custom1
    SortedRows["Duration_batch"] = (SortedRows["Start"] - SortedRows["Finish_DOWN"])/np.timedelta64(1, 'm')
    SortedRows["batch_start"] = np.where(SortedRows["Duration_batch"] < 10,False,True)

    batch_start_filter = SortedRows[SortedRows["batch_start"] == True]

    #useless_clm_remv
    batch_start_filter.drop(columns=["Duration", "Exposure", "Finish_DOWN", "batch_start", "Cluster", "Start", "Finish", "KD","Duration_batch"],inplace=True)

    #batch_nb_add
    index = range(1,len(batch_start_filter)+1)
    batch_start_filter["batch"] = index
    batch_start_filter = batch_start_filter[[
        "Index",
        "batch"
    ]]
    #batch_together
    batch_together = SortedRows.merge(
            batch_start_filter, 
            left_on=["Index"],
            right_on=["Index"],
            suffixes=["","_y"],
            how = 'left'
        )
    remove_tmp_clmns = batch_together.drop(columns=["Duration_batch","Index", "Finish_DOWN", "batch_start"])

    FilledDown = remove_tmp_clmns.copy()
    FilledDown["batch"]= FilledDown["batch"].fillna(method='ffill')

    params = {
        'Duration': 'sum',
        'Start': 'min',
        'Finish': 'max',
        'Exposure': lambda x: True if x.any() else False
    }
    sub = FilledDown[["Cluster", "batch", "KD","Duration","Start","Finish","Exposure"]]
    GroupedRows1 = sub.groupby(["Cluster", "batch", "KD"]).agg(params).reset_index().rename(columns={"Duration":"Usage"})
    return GroupedRows1

In [53]:
FilledDown.columns

Index(['Cluster', 'Start', 'Finish', 'Wafer', 'WBS', 'KD', 'Duration',
       'SC Processing [h]', 'Duration_NormWait [h]', 'Exposure', 'batch'],
      dtype='object')